<a href="https://colab.research.google.com/github/Jucivania-cp/Projeto_Integrado_II/blob/main/Projeto_Integrado_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
!pip install sidrapy

#IMPORTAÇÃO DOS DADOS

##Dados de produção do milho

In [60]:
import pandas as pd
import sidrapy
import requests
import os

# 1. Configurações Iniciais e Códigos dos Municípios (IBGE)
# Lista: Juazeiro, Crato, Barbalha, Jardim, Missão Velha,
# Caririaçu, Farias Brito, Nova Olinda e Santana do Cariri.
municipios_codes = "2307304,2304202,2301901,2307106,2308401,2303204,2304301,2309201,2312106"

def get_base_producao_agricola():
    """Coleta dados da Tabela 839 do SIDRA (Milho) com códigos corrigidos"""
    print("Coletando Base 1: Produção Agrícola (Tabela 839)...")

    # v109 (Plantada), v216 (Colhida), v214 (Produzida), v112 (Rendimento)
    try:
        df_producao = sidrapy.get_table(
                      table_code="839",
                      territorial_level="6",
                      ibge_territorial_code=municipios_codes, # Corrected parameter name
                      period="2010-2021",
                      variable="109,216,214,112"
        )

        # Limpeza de Metadados (Remover a primeira linha que contém descrições)
        df_producao.columns = df_producao.iloc[0]
        df_producao = df_producao.iloc[1:]

        # Converte a coluna Valor para numérico (float64)
        df_producao['Valor'] = pd.to_numeric(df_producao['Valor'], errors='coerce').astype(float)

        # Converte a coluna 'Ano' de inteiro para datetime
        df_producao['Ano'] = pd.to_datetime(df_producao['Ano'], format='%Y')

        #padronisa a coluna Município
        df_producao['Município'] = df_producao['Município'].str.replace(' (CE)', '', regex=False)
        df_producao['Município'] = df_producao['Município'].str.upper().str.strip()

        # Tratamento de valores ausentes e conversão para numérico
        # O SIDRA usa '...' para dados indisponíveis
        # Identificar a coluna de valor
        #value_col_name = df_producao.columns[-1]
        #df_producao[value_col_name] = pd.to_numeric(df_producao[value_col_name].replace(['...', '-'], '0'), errors='coerce')

        df_producao.to_csv("base1_producao_milho_corrigida.csv", index=False)
        print("Base 1 salva com sucesso com códigos 109, 216, 214 e 112.")
        return df_producao

    except Exception as e:
        print(f"Erro na coleta da Tabela 839: {e}")

###Visualização Tabela Produção Agricola

In [61]:
df_producao_cariri = get_base_producao_agricola()

# Filtra as linhas onde a coluna 'Variável' contém exatamente o texto 'Rendimento médio da produção'
df_producao_cariri = df_producao_cariri[df_producao_cariri['Variável'] == 'Rendimento médio da produção']

print("Unidades de Medida:")
print(df_producao_cariri['Unidade de Medida'].unique())
print("ATENÇÃO:\nSe apresentar mais de uma unidade de medida, será preciso padronizar")
print("------------------------------------------------------------------------------")

# Filtra as colunas
df_producao_cariri = df_producao_cariri[['Município (Código)','Município','Ano','Valor']]

# Renomeia as colunas
df_producao_cariri = df_producao_cariri.rename(columns={'Valor': 'Rendimento médio da produção (kg/ha)','Município (Código)': 'ID'})

print("Primeiras 5 linhas:")
display(df_producao_cariri.head())

print("\nÚltimas 5 linhas:")
display(df_producao_cariri.tail())

Coletando Base 1: Produção Agrícola (Tabela 839)...
Base 1 salva com sucesso com códigos 109, 216, 214 e 112.
Unidades de Medida:
['Quilogramas por Hectare']
ATENÇÃO:
Se apresentar mais de uma unidade de medida, será preciso padronizar
------------------------------------------------------------------------------
Primeiras 5 linhas:


,ID,Município,Ano,Rendimento médio da produção (kg/ha)
4,2307304,JUAZEIRO DO NORTE,2010-01-01,448.0
8,2307304,JUAZEIRO DO NORTE,2011-01-01,1700.0
12,2307304,JUAZEIRO DO NORTE,2012-01-01,630.0
16,2307304,JUAZEIRO DO NORTE,2013-01-01,976.0
20,2307304,JUAZEIRO DO NORTE,2014-01-01,3961.0



Últimas 5 linhas:


,ID,Município,Ano,Rendimento médio da produção (kg/ha)
416,2312106,SANTANA DO CARIRI,2017-01-01,2835.0
420,2312106,SANTANA DO CARIRI,2018-01-01,2870.0
424,2312106,SANTANA DO CARIRI,2019-01-01,1600.0
428,2312106,SANTANA DO CARIRI,2020-01-01,2700.0
432,2312106,SANTANA DO CARIRI,2021-01-01,1804.0


In [62]:
#Visualiza se os dados estão no formada adequado para as analises
#df_producao.info()
print(df_producao_cariri.dtypes)
print("-------------------------------------------------")

print("ATENÇÃO:\nAno deve ser datetime64\nRendimento médio da produção deve ser float64")

#Verifica os nomes das cidades
#print("Nome das cidades")
#print(df_producao_cariri['Município'].unique())


0
ID                                              object
Município                                       object
Ano                                     datetime64[ns]
Rendimento médio da produção (kg/ha)           float64
dtype: object
-------------------------------------------------
ATENÇÃO:
Ano deve ser datetime64
Rendimento médio da produção deve ser float64


##Dados do PIB

In [63]:
import requests
import zipfile
import os
import pandas as pd

url_pib = "https://ftp.ibge.gov.br/Pib_Municipios/2022_2023/base/base_de_dados_2010_2023_xlsx.zip"

def baixar_e_extrair_pib_ibge(url_zip, pasta_destino="dados_pib"):
    """
    Baixa a base completa do PIB dos Municípios (2010-2023) e extrai o arquivo Excel.
    """
    arquivo_zip = "pib_2010_2023.zip"

    print(f"Iniciando o download da base IBGE: {url_zip}...")

    try:
        # 1. Faz o download do arquivo ZIP
        response = requests.get(url_zip, stream=True, verify=False)
        response.raise_for_status()

        with open(arquivo_zip, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024*1024): # 1MB por vez
                if chunk:
                    f.write(chunk)

        print("Download concluído! Extraindo a base de dados...")

        # 2. Extrai o conteúdo do ZIP
        if not os.path.exists(pasta_destino):
            os.makedirs(pasta_destino)

        with zipfile.ZipFile(arquivo_zip, 'r') as zip_ref:
            zip_ref.extractall(pasta_destino)

        # 3. Limpeza: remove o arquivo comprimido
        os.remove(arquivo_zip)

        print(f"Base de dados extraída com sucesso na pasta: '{pasta_destino}'")
        return pasta_destino

    except Exception as e:
        print(f"Erro no download da base PIB: {e}")
        return None

#
pasta_pib = baixar_e_extrair_pib_ibge(url_pib)

if pasta_pib:
    # Identifica o arquivo Excel extraído (geralmente tem nome similar ao ZIP)
    arquivos = os.listdir(pasta_pib)
    caminho_excel = os.path.join(pasta_pib, arquivos[0]) # Pega o primeiro arquivo extraído da lista

    # Carrega e filtra apenas os municípios do Cariri para otimizar o projeto
    print(f"Carregando partir de {caminho_excel}...")
    df_pib_bruto = pd.read_excel(caminho_excel)

    # Converte a coluna 'Ano' de inteiro para datetime
    df_pib_bruto['Ano'] = pd.to_datetime(df_pib_bruto['Ano'], format='%Y')

    #Padroniza o nome do municipio
    df_pib_bruto['Nome do Município'] = df_pib_bruto['Nome do Município'].str.upper().str.strip()

    df_pib_bruto = df_pib_bruto.rename(columns={'Nome do Município': 'Município',
                                                'Código do Município': 'ID',
                                                'Valor adicionado bruto da Agropecuária, \na preços correntes\n(R$ 1.000)': 'Valor adicionado bruto da Agropecuária (R$ 1.000)',
                                                'Valor adicionado bruto total, \na preços correntes\n(R$ 1.000)': 'Valor adicionado bruto total (R$ 1.000)'})


Iniciando o download da base IBGE: https://ftp.ibge.gov.br/Pib_Municipios/2022_2023/base/base_de_dados_2010_2023_xlsx.zip...


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ftp.ibge.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Download concluído! Extraindo a base de dados...
Base de dados extraída com sucesso na pasta: 'dados_pib'
Carregando partir de dados_pib/PIB dos Municípios - base de dados 2010-2023.xlsx...


###Visualizar e filtrar tabela PIB

In [64]:
# Filtra as colunas
df_pib_bruto = df_pib_bruto[['ID',
                             'Município',
                             'Ano',
                             'Valor adicionado bruto da Agropecuária (R$ 1.000)',
                             'Valor adicionado bruto total (R$ 1.000)']]

# Visualização de nomes unicos
#print(df_producao['Variável'].unique())

# Lista de códigos IBGE dos seus 9 cidades do Cariri (convertidos para int)
municipios_codes_list = [int(code) for code in ['2307304','2304202','2301901','2307106','2308401','2303204','2304301','2309201','2312106']]

# Filtra as linhas onde a coluna 'Código do Município' está na lista de municípios do Cariri
df_pib_cariri = df_pib_bruto[df_pib_bruto['ID'].isin(municipios_codes_list)]

df_pib_cariri = df_pib_cariri[(df_pib_cariri['Ano'].dt.year >= 2010) & (df_pib_cariri['Ano'].dt.year <= 2021)]
print("Primeiras 5 linhas:")
display(df_pib_cariri.head())

print("\nÚltimas 5 linhas:")
display(df_pib_cariri.tail())

Primeiras 5 linhas:


,ID,Município,Ano,Valor adicionado bruto da Agropecuária (R$ 1.000),Valor adicionado bruto total (R$ 1.000)
912,2301901,BARBALHA,2010-01-01,22325.115,417550.919
927,2303204,CARIRIAÇU,2010-01-01,10879.484,96216.676
940,2304202,CRATO,2010-01-01,36135.178,894329.801
946,2304301,FARIAS BRITO,2010-01-01,8210.129,71915.151
985,2307106,JARDIM,2010-01-01,18184.741,104026.703



Últimas 5 linhas:


,ID,Município,Ano,Valor adicionado bruto da Agropecuária (R$ 1.000),Valor adicionado bruto total (R$ 1.000)
62241,2307106,JARDIM,2021-01-01,67179.809,281656.825
62244,2307304,JUAZEIRO DO NORTE,2021-01-01,31924.965,4492875.236
62259,2308401,MISSÃO VELHA,2021-01-01,198475.570,495645.087
62267,2309201,NOVA OLINDA,2021-01-01,16967.776,154267.978
62304,2312106,SANTANA DO CARIRI,2021-01-01,28382.623,153777.873


In [65]:
df_pib_cariri.dtypes

,0
ID,int64
Município,object
Ano,datetime64[ns]
Valor adicionado bruto da Agropecuária (R$ 1.000),float64
Valor adicionado bruto total (R$ 1.000),float64


##Dados Pluviometria

In [66]:
import pandas as pd
import requests
import zipfile
import os

url = "https://cdn.funceme.br/calendario/postos/postos.zip"

def baixar_e_extrair_funceme(url_zip, pasta_destino="dados_funceme"):
    """
    Baixa o arquivo ZIP da FUNCEME e extrai seu conteúdo automaticamente.
    """
    arquivo_zip = "postos.zip"

    print(f"Iniciando o download de: {url_zip}...")

    try:
        # 1. Faz o download do arquivo
        response = requests.get(url_zip, stream=True)
        response.raise_for_status() # Verifica se o link está ativo

        with open(arquivo_zip, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        print("Download concluído! Extraindo arquivos...")

        # 2. Extrai o conteúdo do ZIP
        if not os.path.exists(pasta_destino):
            os.makedirs(pasta_destino)

        with zipfile.ZipFile(arquivo_zip, 'r') as zip_ref:
            zip_ref.extractall(pasta_destino)

        # 3. Limpeza: remove o arquivo .zip após extrair
        os.remove(arquivo_zip)

        print(f"Arquivos extraídos com sucesso na pasta: '{pasta_destino}'")
        return pasta_destino

    except Exception as e:
        print(f"Erro ao automatizar o download: {e}")
        return None

# Execução
pasta_dos_csvs = baixar_e_extrair_funceme(url)

def processar_arquivos_por_id(caminho_pasta):
    # Dicionário de mapeamento (Exemplo: preencha com os IDs que você encontrar no site)
    # Formato: 'ID_DO_ARQUIVO': ['NOME_DO_MUNICIPIO', CODIGO_IBGE]
    mapeamento_cariri = {
        '78': ['JUAZEIRO DO NORTE', 2307304],
        '43': ['CRATO', 2304202],
        '20': ['BARBALHA', 2301901],
        '76': ['JARDIM', 2307106],
        '91': ['MISSAO VELHA', 2308401],
        '33': ['CARIRIACU', 2303204],
        '45': ['FARIAS BRITO', 2304301],
        '99': ['NOVA OLINDA', 2309201],
        '131': ['SANTANA DO CARIRI', 2312106]
        }

    lista_dfs = []

    print("Iniciando leitura dos arquivos por ID...")

    for id_posto, nome_cidade in mapeamento_cariri.items():
        arquivo_nome = f"{id_posto}.txt"
        caminho_completo = os.path.join(caminho_pasta, arquivo_nome)

        if os.path.exists(caminho_completo):
            # Ler o TXT
            df_temp = pd.read_csv(caminho_completo, sep=';', encoding='latin-1')

            if 'Total' in df_temp.columns:
              df_temp = df_temp.drop(columns=['Total'])

            # Diagnóstico: Imprimir os nomes das colunas
            print(f"Colunas em {arquivo_nome} ({nome_cidade[0]}): {df_temp.columns.tolist()}")

            # Adicionar coluna com o nome do município para identificação
            df_temp['Municipio'] = nome_cidade[0] # Assign only the municipality name
            lista_dfs.append(df_temp)
            print(f"Arquivo {arquivo_nome} ({nome_cidade[0]}) carregado.")
        else:
            print(f"Aviso: Arquivo {arquivo_nome} não encontrado para {nome_cidade[0]}.")

    if lista_dfs:
        df_raw = pd.concat(lista_dfs, ignore_index=True)

        # Junta as colunas 'Dia' em uma única coluna.
        id_vars = ['Municipios', 'Postos', 'Latitude', 'Longitude', 'Anos', 'Meses', 'Municipio']
        value_vars = [col for col in df_raw.columns if col.startswith('Dia') and col[3:].isdigit()]

        df_melted = df_raw.melt(
            id_vars=id_vars,
            value_vars=value_vars,
            var_name='Dia_str',
            value_name='valor_chuva_mm'
        )

        # Substitui 999,0, -999 e -999,0 por NaN para `valor_chuva_mm`
        df_melted['valor_chuva_mm'] = df_melted['valor_chuva_mm'].replace([999.0, -999, -999.0], pd.NA)

        # Extrair o número do dia de 'Dia_str' (por exemplo, 'Dia1' -> '1')
        df_melted['Dia'] = df_melted['Dia_str'].str.replace('Dia', '', regex=False).astype(int)

        # Criar coluna 'Dados'
        # Preencher mês e dia com zeros à esquerda para a correta análise da data
        df_melted['Data'] = pd.to_datetime(
            df_melted['Anos'].astype(str) + '-' +
            df_melted['Meses'].astype(str).str.zfill(2) + '-' +
            df_melted['Dia'].astype(str).str.zfill(2),
            errors='coerce' # Coerce errors to NaT for invalid dates (e.g., Feb 30)
        )

        # Filtrar linhas com datas inválidas (NaT)
        df_melted.dropna(subset=['Data'], inplace=True)

        # Filtra periodo de 2010 a 2021
        df_final_local = df_melted[(df_melted['Data'].dt.year >= 2010) & (df_melted['Data'].dt.year <= 2021)]

        # Converte a coluna Valor para numérico (float64)
        df_final_local['valor_chuva_mm'] = pd.to_numeric(df_final_local['valor_chuva_mm'], errors='coerce').astype(float)

        #Renomeia colunas
        df_final_local = df_final_local.rename(columns={'Municipio': 'Município'})

        # Seleciona e ordena as colunas
        df_final_local = df_final_local[['Município','Data', 'valor_chuva_mm']]

        # Adiciona a coluna ID_IBGE para facilitar o merge posterior
        # Mapeamento do nome do município para o ID_IBGE, buscando o ID do dicionário original
        ibge_mapping = {v[0].upper(): v[1] for k, v in mapeamento_cariri.items()} # Corrected ibge_mapping creation
        df_final_local['ID'] = df_final_local['Município'].map(ibge_mapping).astype(int)

        return df_final_local
    else:
        return None

#Processamento dos arquivos
if pasta_dos_csvs:
    # Caso os CSVs estejam em uma subpasta 'postos' dentro da extração:
    caminho_real = os.path.join(pasta_dos_csvs, "postos")
    if not os.path.exists(caminho_real):
        caminho_real = pasta_dos_csvs # Se não houver subpasta, usa a principal

    df_pluviometria_diaria = processar_arquivos_por_id(caminho_real)

    #if df_final is not None:
        #print("\nBase Pluviométrica do Cariri gerada com sucesso!")
        #display(df_final.head())
    #else:
        #print("\nErro: Nenhum dado foi processado. Verifique os IDs e o caminho dos arquivos.")

Iniciando o download de: https://cdn.funceme.br/calendario/postos/postos.zip...
Download concluído! Extraindo arquivos...
Arquivos extraídos com sucesso na pasta: 'dados_funceme'
Iniciando leitura dos arquivos por ID...
Colunas em 78.txt (JUAZEIRO DO NORTE): ['Municipios', 'Postos', 'Latitude', 'Longitude', 'Anos', 'Meses', 'Dia1', 'Dia2', 'Dia3', 'Dia4', 'Dia5', 'Dia6', 'Dia7', 'Dia8', 'Dia9', 'Dia10', 'Dia11', 'Dia12', 'Dia13', 'Dia14', 'Dia15', 'Dia16', 'Dia17', 'Dia18', 'Dia19', 'Dia20', 'Dia21', 'Dia22', 'Dia23', 'Dia24', 'Dia25', 'Dia26', 'Dia27', 'Dia28', 'Dia29', 'Dia30', 'Dia31']
Arquivo 78.txt (JUAZEIRO DO NORTE) carregado.
Colunas em 43.txt (CRATO): ['Municipios', 'Postos', 'Latitude', 'Longitude', 'Anos', 'Meses', 'Dia1', 'Dia2', 'Dia3', 'Dia4', 'Dia5', 'Dia6', 'Dia7', 'Dia8', 'Dia9', 'Dia10', 'Dia11', 'Dia12', 'Dia13', 'Dia14', 'Dia15', 'Dia16', 'Dia17', 'Dia18', 'Dia19', 'Dia20', 'Dia21', 'Dia22', 'Dia23', 'Dia24', 'Dia25', 'Dia26', 'Dia27', 'Dia28', 'Dia29', 'Dia30', 'Di

/tmp/ipython-input-3513345871.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_local['valor_chuva_mm'] = pd.to_numeric(df_final_local['valor_chuva_mm'], errors='coerce').astype(float)


In [67]:
print("Primeiras 5 linhas:")
display(df_pluviometria_diaria.head())

print("\nÚltimas 5 linhas:")
display(df_pluviometria_diaria.tail())

Primeiras 5 linhas:


,Município,Data,valor_chuva_mm,ID
432,JUAZEIRO DO NORTE,2010-01-01,54.5,2307304
433,JUAZEIRO DO NORTE,2010-02-01,0.0,2307304
434,JUAZEIRO DO NORTE,2010-03-01,0.0,2307304
435,JUAZEIRO DO NORTE,2010-04-01,0.0,2307304
436,JUAZEIRO DO NORTE,2010-05-01,0.0,2307304



Últimas 5 linhas:


,Município,Data,valor_chuva_mm,ID
168550,SANTANA DO CARIRI,2020-12-31,0.0,2312106
168551,SANTANA DO CARIRI,2021-01-31,3.2,2312106
168553,SANTANA DO CARIRI,2021-03-31,0.0,2312106
168556,SANTANA DO CARIRI,2021-10-31,0.0,2312106
168558,SANTANA DO CARIRI,2021-12-31,0.0,2312106


In [68]:
df_pluviometria_diaria.dtypes

,0
Município,object
Data,datetime64[ns]
valor_chuva_mm,float64
ID,int64


###Agregação Anual (FUNCEME)

In [69]:
import pandas as pd
import numpy as np

def agregar_dados_anuais(df_pluviometria_diaria):
    df = df_pluviometria_diaria.copy()
    df['Precipitacao'] = pd.to_numeric(df['valor_chuva_mm'], errors='coerce').fillna(0)
    df['Ano'] = df['Data'].dt.year

    # Depuração: Imprima as colunas antes do agrupamento para verificar se há 'Precipitação'.
    print("Columns before groupby:", df.columns)

    # --- PASSO 1: CONSOLIDAÇÃO DIÁRIA (Média dos postos por dia) ---
    # Isso reduz as múltiplas linhas diárias de Barbalha para uma única média por dia
    df_diario_consolidado = df.groupby(['Município', 'Data', 'Ano', 'ID']).agg(
        Precipitacao_Media_Dia=('Precipitacao', 'mean')
    ).reset_index()

    # Impressão de diagnóstico: Verifica colunas de df_diario_consolidado
    print(f"Columns in df_diario_consolidado: {df_diario_consolidado.columns.tolist()}")
    print(f"Head of df_diario_consolidado:\n{df_diario_consolidado.head()}")

    # AGREGAÇÃO ANUAL (Soma sobre a média diária)
    df_anual = df_diario_consolidado.groupby(['Município', 'Ano']).agg(
        ID=('ID', 'first'),
        Precipitacao_Total=('Precipitacao_Media_Dia', 'sum'), # Soma das médias diárias
        Media_Diaria=('Precipitacao_Media_Dia', 'mean'),
        Dias_Com_Chuva=('Precipitacao_Media_Dia', lambda x: (x >= 2.0).sum())
    ).reset_index()

    # Lógica de veranicos usando a base já consolidada
    # intervalo de meses para [3,4,5,6,7]
    df_quadra = df_diario_consolidado[df_diario_consolidado['Data'].dt.month.isin([1,2,3,4,5])].copy()

    def contar_veranicos(serie):
        dias_secos = serie < 2.0
        grupos = (dias_secos != dias_secos.shift()).cumsum()
        return (dias_secos[dias_secos].groupby(grupos[dias_secos]).size() >= 5).sum()

    veranicos = df_quadra.groupby(['Município', 'Ano'])['Precipitacao_Media_Dia'].apply(contar_veranicos).reset_index()
    veranicos.rename(columns={'Precipitacao_Media_Dia': 'Qtd_Veranicos'}, inplace=True)

    df_final = pd.merge(df_anual, veranicos, on=['Município', 'Ano'], how='left').fillna(0)
    df_final['Ano'] = pd.to_datetime(df_final['Ano'], format='%Y')

    return df_final, df_diario_consolidado

###Visualização Dados Pluviometria

In [70]:
df_pluviometria_anual, df_diario_consolidado = agregar_dados_anuais(df_pluviometria_diaria)

print("Primeiras 5 linhas:")
display(df_pluviometria_anual.head())

print("\nÚltimas 5 linhas:")
display(df_pluviometria_anual.tail())

Columns before groupby: Index(['Município', 'Data', 'valor_chuva_mm', 'ID', 'Precipitacao', 'Ano'], dtype='object')
Columns in df_diario_consolidado: ['Município', 'Data', 'Ano', 'ID', 'Precipitacao_Media_Dia']
Head of df_diario_consolidado:
  Município       Data   Ano       ID  Precipitacao_Media_Dia
0  BARBALHA 2010-01-01  2010  2301901                    55.0
1  BARBALHA 2010-01-02  2010  2301901                    28.5
2  BARBALHA 2010-01-03  2010  2301901                    34.0
3  BARBALHA 2010-01-04  2010  2301901                     9.0
4  BARBALHA 2010-01-05  2010  2301901                     0.0
Primeiras 5 linhas:


,Município,Ano,ID,Precipitacao_Total,Media_Diaria,Dias_Com_Chuva,Qtd_Veranicos
0,BARBALHA,2010-01-01,2301901,1059.9,2.903836,49,10
1,BARBALHA,2011-01-01,2301901,1431.2,3.921096,59,9
2,BARBALHA,2012-01-01,2301901,540.5,1.476776,40,7
3,BARBALHA,2013-01-01,2301901,1238.1,3.392055,57,10
4,BARBALHA,2014-01-01,2301901,1209.1,3.312603,63,7



Últimas 5 linhas:


,Município,Ano,ID,Precipitacao_Total,Media_Diaria,Dias_Com_Chuva,Qtd_Veranicos
103,SANTANA DO CARIRI,2017-01-01,2312106,515.9,1.413425,34,11
104,SANTANA DO CARIRI,2018-01-01,2312106,1097.1,3.005753,55,9
105,SANTANA DO CARIRI,2019-01-01,2312106,1101.4,3.017534,53,8
106,SANTANA DO CARIRI,2020-01-01,2312106,915.5,2.501366,35,5
107,SANTANA DO CARIRI,2021-01-01,2312106,562.1,2.322727,41,7


In [71]:
#Verifica se os dados estão no fomato adequado para as analises
#df_pluviometria_anual.info()
df_pluviometria_anual.dtypes

#Verifica os nomes das cidades
#print(df_pluviometria_anual['Município'].unique())

,0
Município,object
Ano,datetime64[ns]
ID,int64
Precipitacao_Total,float64
Media_Diaria,float64
Dias_Com_Chuva,int64
Qtd_Veranicos,int64


#AGREGAÇÃO DAS BASES

In [72]:
import pandas as pd

def consolidar_projeto_integrado(df_producao_cariri, df_pib_cariri, df_pluviometria_anual):
    print("Iniciando a unificação das bases (Merge)...")

    # Verifique antes se ID em df_producao_cariri é do tipo inteiro
    df_producao_cariri_copy = df_producao_cariri.copy()
    df_producao_cariri_copy['ID'] = pd.to_numeric(df_producao_cariri_copy['ID'], errors='coerce').astype(int)

    # 1. Unir Produção Agrícola com Pluviometria Anual
    base_consolidada = pd.merge(
        df_producao_cariri_copy,
        df_pluviometria_anual,
        left_on=['ID', 'Ano'],
        right_on=['ID', 'Ano'],
        how='inner',
        suffixes=('_prod', '_pluvio')
    )

    # Renomeie a coluna de produção 'Município' e remova a coluna de pluviometria 'Município'.
    if 'Município_prod' in base_consolidada.columns:
        base_consolidada = base_consolidada.rename(columns={'Município_prod': 'Município'})
    if 'Município_pluvio' in base_consolidada.columns:
        base_consolidada = base_consolidada.drop(columns=['Município_pluvio'])

    # 2. Unir com a base de PIB
    base_final = pd.merge(
        base_consolidada,
        df_pib_cariri[['ID', 'Ano', 'Valor adicionado bruto da Agropecuária (R$ 1.000)', 'Valor adicionado bruto total (R$ 1.000)']],
        left_on=['ID', 'Ano'],
        right_on=['ID', 'Ano'],
        how='inner'
    )

    print(f"Merge concluído! Base final gerada com {base_final.shape} linhas.")
    return base_final

# Agregação das bases
df_projeto_cariri = consolidar_projeto_integrado(df_producao_cariri, df_pib_cariri, df_pluviometria_anual)

#Renomeia colunas
df_projeto_cariri = df_projeto_cariri.rename(columns={'Precipitacao_Total': 'Precipitação Total (mm)',
                                                'Media_Diaria': 'Precipitação Média (mm)',
                                                'Dias_Com_Chuva': 'Dias Chuvosos',
                                                'Qtd_Veranicos': 'Quantidade de Veranicos'})


print("Primeiras 5 linhas:")
display(df_projeto_cariri.head())

print("\nÚltimas 5 linhas:")
display(df_projeto_cariri.tail())

Iniciando a unificação das bases (Merge)...
Merge concluído! Base final gerada com (108, 10) linhas.
Primeiras 5 linhas:


,ID,Município,Ano,Rendimento médio da produção (kg/ha),Precipitação Total (mm),Precipitação Média (mm),Dias Chuvosos,Quantidade de Veranicos,Valor adicionado bruto da Agropecuária (R$ 1.000),Valor adicionado bruto total (R$ 1.000)
0,2307304,JUAZEIRO DO NORTE,2010-01-01,448.0,793.5,2.173973,33,7,9783.057,1922091.144
1,2307304,JUAZEIRO DO NORTE,2011-01-01,1700.0,1529.0,4.189041,43,9,12982.403,2210242.426
2,2307304,JUAZEIRO DO NORTE,2012-01-01,630.0,520.5,1.422131,31,8,10814.307,2511495.741
3,2307304,JUAZEIRO DO NORTE,2013-01-01,976.0,1021.0,2.797260,42,10,13716.955,2865169.099
4,2307304,JUAZEIRO DO NORTE,2014-01-01,3961.0,1022.5,2.801370,44,8,18927.194,3416392.060



Últimas 5 linhas:


,ID,Município,Ano,Rendimento médio da produção (kg/ha),Precipitação Total (mm),Precipitação Média (mm),Dias Chuvosos,Quantidade de Veranicos,Valor adicionado bruto da Agropecuária (R$ 1.000),Valor adicionado bruto total (R$ 1.000)
103,2312106,SANTANA DO CARIRI,2017-01-01,2835.0,515.9,1.413425,34,11,22111.393,128693.253
104,2312106,SANTANA DO CARIRI,2018-01-01,2870.0,1097.1,3.005753,55,9,14498.481,120093.729
105,2312106,SANTANA DO CARIRI,2019-01-01,1600.0,1101.4,3.017534,53,8,14997.691,129938.538
106,2312106,SANTANA DO CARIRI,2020-01-01,2700.0,915.5,2.501366,35,5,24541.406,145991.594
107,2312106,SANTANA DO CARIRI,2021-01-01,1804.0,562.1,2.322727,41,7,28382.623,153777.873


In [73]:
def verifica_dados_ausentes():
  """Verifica se existem dados ausentes na base de dados"""
  print("Verificando dados ausentes...")
  print("\n")
  print(df_producao_cariri.isnull().sum())
  print("\n")
  print(df_producao_cariri[df_producao_cariri.isnull().any(axis=1)])

verifica_dados_ausentes()

Verificando dados ausentes...


0
ID                                      0
Município                               0
Ano                                     0
Rendimento médio da produção (kg/ha)    1
dtype: int64


0         ID Município        Ano  Rendimento médio da produção (kg/ha)
156  2307106    JARDIM 2012-01-01                                   NaN


In [74]:
df_projeto_cariri.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 10 columns):
 #   Column                                             Non-Null Count  Dtype         
---  ------                                             --------------  -----         
 0   ID                                                 108 non-null    int64         
 1   Município                                          108 non-null    object        
 2   Ano                                                108 non-null    datetime64[ns]
 3   Rendimento médio da produção (kg/ha)               107 non-null    float64       
 4   Precipitação Total (mm)                            108 non-null    float64       
 5   Precipitação Média (mm)                            108 non-null    float64       
 6   Dias Chuvosos                                      108 non-null    int64         
 7   Quantidade de Veranicos                            108 non-null    int64         
 8   Valor adicionado bru